In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.utils import resample
from sklearn.feature_selection import VarianceThreshold
from sklearn.feature_selection import SelectKBest, f_classif
# import tensorflow as tf
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import Dense, LeakyReLU, BatchNormalization, Dropout

In [3]:
df1 = pd.read_csv("D:/minor_project/datasets/CICIDS_dataset/Monday-WorkingHours.pcap_ISCX.csv")
df2 = pd.read_csv("D:/minor_project/datasets/CICIDS_dataset/Tuesday-WorkingHours.pcap_ISCX.csv")
df3 = pd.read_csv("D:/minor_project/datasets/CICIDS_dataset/Wednesday-workingHours.pcap_ISCX.csv")
df4 = pd.read_csv("D:/minor_project/datasets/CICIDS_dataset/Thursday-WorkingHours-Morning-WebAttacks.pcap_ISCX.csv")
df5 = pd.read_csv("D:/minor_project/datasets/CICIDS_dataset/Thursday-WorkingHours-Afternoon-Infilteration.pcap_ISCX.csv")
df7 = pd.read_csv("D:/minor_project/datasets/CICIDS_dataset/Friday-WorkingHours-Afternoon-DDos.pcap_ISCX.csv")
df8 = pd.read_csv("D:/minor_project/datasets/CICIDS_dataset/Friday-WorkingHours-Afternoon-PortScan.pcap_ISCX.csv")
df6 = pd.read_csv("D:/minor_project/datasets/CICIDS_dataset/Friday-WorkingHours-Morning.pcap_ISCX.csv")

In [21]:
# Merge all parts
df = pd.concat([df1, df2, df3, df4,df5,df6,df7,df8], ignore_index=True)

In [22]:
df.shape

(2830743, 79)

In [23]:
df.columns

Index([' Destination Port', ' Flow Duration', ' Total Fwd Packets',
       ' Total Backward Packets', 'Total Length of Fwd Packets',
       ' Total Length of Bwd Packets', ' Fwd Packet Length Max',
       ' Fwd Packet Length Min', ' Fwd Packet Length Mean',
       ' Fwd Packet Length Std', 'Bwd Packet Length Max',
       ' Bwd Packet Length Min', ' Bwd Packet Length Mean',
       ' Bwd Packet Length Std', 'Flow Bytes/s', ' Flow Packets/s',
       ' Flow IAT Mean', ' Flow IAT Std', ' Flow IAT Max', ' Flow IAT Min',
       'Fwd IAT Total', ' Fwd IAT Mean', ' Fwd IAT Std', ' Fwd IAT Max',
       ' Fwd IAT Min', 'Bwd IAT Total', ' Bwd IAT Mean', ' Bwd IAT Std',
       ' Bwd IAT Max', ' Bwd IAT Min', 'Fwd PSH Flags', ' Bwd PSH Flags',
       ' Fwd URG Flags', ' Bwd URG Flags', ' Fwd Header Length',
       ' Bwd Header Length', 'Fwd Packets/s', ' Bwd Packets/s',
       ' Min Packet Length', ' Max Packet Length', ' Packet Length Mean',
       ' Packet Length Std', ' Packet Length Variance', '

In [24]:
pd.set_option('display.maX_columns', None)

In [25]:
# Strip spaces from column names
df.columns = df.columns.str.strip()

# Now access unique values from 'Label'
unique_values = df["Label"].unique()
print(unique_values)


['BENIGN' 'FTP-Patator' 'SSH-Patator' 'DoS slowloris' 'DoS Slowhttptest'
 'DoS Hulk' 'DoS GoldenEye' 'Heartbleed' 'Web Attack � Brute Force'
 'Web Attack � XSS' 'Web Attack � Sql Injection' 'Infiltration' 'Bot'
 'DDoS' 'PortScan']


In [26]:
df['Label'].value_counts()

Label
BENIGN                        2273097
DoS Hulk                       231073
PortScan                       158930
DDoS                           128027
DoS GoldenEye                   10293
FTP-Patator                      7938
SSH-Patator                      5897
DoS slowloris                    5796
DoS Slowhttptest                 5499
Bot                              1966
Web Attack � Brute Force         1507
Web Attack � XSS                  652
Infiltration                       36
Web Attack � Sql Injection         21
Heartbleed                         11
Name: count, dtype: int64

In [27]:
# Find columns where all values are same
unique_col=[]
for col in df.columns:
    if len(df[col].unique())==1:
        unique_col.append(col)
        

print(unique_col)

['Bwd PSH Flags', 'Bwd URG Flags', 'Fwd Avg Bytes/Bulk', 'Fwd Avg Packets/Bulk', 'Fwd Avg Bulk Rate', 'Bwd Avg Bytes/Bulk', 'Bwd Avg Packets/Bulk', 'Bwd Avg Bulk Rate']


In [28]:
# for col in df.columns:
#     print(len(df[col].unique()))


In [29]:
df = df.drop(columns=unique_col,errors='ignore')

In [30]:
print(df.shape)

(2830743, 71)


In [31]:
df['Label'].value_counts()

Label
BENIGN                        2273097
DoS Hulk                       231073
PortScan                       158930
DDoS                           128027
DoS GoldenEye                   10293
FTP-Patator                      7938
SSH-Patator                      5897
DoS slowloris                    5796
DoS Slowhttptest                 5499
Bot                              1966
Web Attack � Brute Force         1507
Web Attack � XSS                  652
Infiltration                       36
Web Attack � Sql Injection         21
Heartbleed                         11
Name: count, dtype: int64

In [15]:

# df['Label'].value_counts()

In [17]:
import pandas as pd

# Define the class mapping
class_mapping = {
    'BENIGN': 'BENIGN',
    'Bot': 'Botnet',
    'FTP-Patator': 'Brute Force',
    'SSH-Patator': 'Brute Force',
    'DDoS': 'DDoS',
    'DoS': 'DoS',
    'DoS GoldenEye': 'DoS',
    'DoS Hulk': 'DoS',
    'DoS Slowhttptest': 'DoS',
    'DoS Slowloris': 'DoS',
    'Heartbleed': 'DoS',
    'PortScan': 'Probe',
    'Web Attack: Brute Force': 'Web Attack',
    'Web Attack: SQL Injection': 'Web Attack',
    'Web Attack: XSS': 'Web Attack',
    'Infiltration': 'Infiltration'
}

# Sample DataFrame creation (replace this with your actual DataFrame)
data = {
    'Label': [
        'BENIGN', 'DoS Hulk', 'PortScan', 'DDoS', 'DoS GoldenEye',
        'FTP-Patator', 'SSH-Patator', 'DoS slowloris', 'DoS Slowhttptest',
        'Bot', 'Web Attack: Brute Force', 'Web Attack: XSS', 
        'Infiltration', 'Web Attack: SQL Injection', 'Heartbleed'
    ],
    'Count': [2273097, 231073, 158930, 128027, 10293, 7938, 5897, 5796, 5499, 1966, 1507, 652, 36, 21, 11]
}

df = pd.DataFrame(data)

# Make a copy to avoid modifying the original DataFrame
df = df.copy()

# Convert the labels using the mapping
df["Mapped Label"] = df["Label"].map(class_mapping)

# Check for any labels that didn't get mapped
if df["Mapped Label"].isnull().any():
    unmapped = df[df["Mapped Label"].isnull()]["Label"].unique()
    print(f"Warning: The following original classes were not mapped: {unmapped}")

# Print the updated DataFrame
print(df)

                        Label    Count  Mapped Label
0                      BENIGN  2273097        BENIGN
1                    DoS Hulk   231073           DoS
2                    PortScan   158930         Probe
3                        DDoS   128027          DDoS
4               DoS GoldenEye    10293           DoS
5                 FTP-Patator     7938   Brute Force
6                 SSH-Patator     5897   Brute Force
7               DoS slowloris     5796           NaN
8            DoS Slowhttptest     5499           DoS
9                         Bot     1966        Botnet
10    Web Attack: Brute Force     1507    Web Attack
11            Web Attack: XSS      652    Web Attack
12               Infiltration       36  Infiltration
13  Web Attack: SQL Injection       21    Web Attack
14                 Heartbleed       11           DoS


In [32]:
dfc = df.copy()

In [33]:
dfc.shape

(2830743, 71)

In [34]:
dfc['Label'].value_counts()

Label
BENIGN                        2273097
DoS Hulk                       231073
PortScan                       158930
DDoS                           128027
DoS GoldenEye                   10293
FTP-Patator                      7938
SSH-Patator                      5897
DoS slowloris                    5796
DoS Slowhttptest                 5499
Bot                              1966
Web Attack � Brute Force         1507
Web Attack � XSS                  652
Infiltration                       36
Web Attack � Sql Injection         21
Heartbleed                         11
Name: count, dtype: int64

In [42]:
filtered_df = dfc[~dfc["Label"].isin(["Heartbleed"])]


In [43]:
filtered_df['Label'].value_counts()

Label
BENIGN                        2273097
DoS Hulk                       231073
PortScan                       158930
DDoS                           128027
DoS GoldenEye                   10293
FTP-Patator                      7938
SSH-Patator                      5897
DoS slowloris                    5796
DoS Slowhttptest                 5499
Bot                              1966
Web Attack � Brute Force         1507
Web Attack � XSS                  652
Infiltration                       36
Web Attack � Sql Injection         21
Name: count, dtype: int64

In [44]:
# Step 1: Handle missing values
filtered_df.replace(['?', 'Infinity', np.inf, -np.inf], np.nan, inplace=True)
filtered_df.dropna(inplace=True)

C:\Users\hp\AppData\Local\Temp\ipykernel_9308\4213380657.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df.replace(['?', 'Infinity', np.inf, -np.inf], np.nan, inplace=True)
C:\Users\hp\AppData\Local\Temp\ipykernel_9308\4213380657.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df.dropna(inplace=True)


In [45]:
filtered_df.shape

(2827865, 71)

In [46]:
filtered_df['Label'].value_counts()

Label
BENIGN                        2271320
DoS Hulk                       230124
PortScan                       158804
DDoS                           128025
DoS GoldenEye                   10293
FTP-Patator                      7935
SSH-Patator                      5897
DoS slowloris                    5796
DoS Slowhttptest                 5499
Bot                              1956
Web Attack � Brute Force         1507
Web Attack � XSS                  652
Infiltration                       36
Web Attack � Sql Injection         21
Name: count, dtype: int64

In [51]:
def map_attack(label):
    # Match "DoS" attacks but avoid converting "DDoS"
    if 'Infiltration' in label:
        return 'DoS'
    elif 'Patator' in label:
        return 'Brute Force'
    elif 'Web Attack' in label:
        return 'Web Attack'
    else:
        return label  # If there's any label that doesn't fit, keep the original

# Apply the function to the 'Label' column
filtered_df['Label'] = filtered_df['Label'].apply(map_attack)

C:\Users\hp\AppData\Local\Temp\ipykernel_9308\3716215305.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['Label'] = filtered_df['Label'].apply(map_attack)


In [52]:
filtered_df['Label'].value_counts()

Label
BENIGN         2271320
DoS             251748
PortScan        158804
DDoS            128025
Brute Force      13832
Web Attack        2180
Bot               1956
Name: count, dtype: int64

In [53]:
# 2. Encode Categorical Columns (if any)
cat_cols = filtered_df.select_dtypes(include=['object']).columns.tolist()
cat_cols = [col for col in cat_cols if col != "Label"]  # skip target column

print(cat_cols)


[]


In [54]:
label_encoders = {}
for col in cat_cols:
    le = LabelEncoder()
    filtered_df[col] = le.fit_transform(filtered_df[col])
    label_encoders[col] = le

In [55]:
# 4. Encode Target Column (category)
label_encoder = LabelEncoder()
filtered_df['Lable_encoded'] = label_encoder.fit_transform(filtered_df["Label"])


C:\Users\hp\AppData\Local\Temp\ipykernel_9308\1144850539.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['Lable_encoded'] = label_encoder.fit_transform(filtered_df["Label"])


In [56]:
# Save mapping for interpretability
attack_mapping = dict(zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_)))
print("Attack Category Encoding Mapping:")
for attack, code in attack_mapping.items():
    print(f"{attack}: {code}")



Attack Category Encoding Mapping:
BENIGN: 0
Bot: 1
Brute Force: 2
DDoS: 3
DoS: 4
PortScan: 5
Web Attack: 6


In [57]:
filtered_df.shape

(2827865, 72)

In [58]:
filtered_df['Label'].value_counts()

Label
BENIGN         2271320
DoS             251748
PortScan        158804
DDoS            128025
Brute Force      13832
Web Attack        2180
Bot               1956
Name: count, dtype: int64

In [59]:
filtered_df["Lable_encoded"].value_counts()

Lable_encoded
0    2271320
4     251748
5     158804
3     128025
2      13832
6       2180
1       1956
Name: count, dtype: int64

In [60]:
filtered_df = filtered_df.drop(columns=['Label'], errors='ignore')

In [61]:
# Define int32 range
int32_min = np.iinfo(np.int32).min
int32_max = np.iinfo(np.int32).max
df_filtered = filtered_df[(filtered_df.ge(int32_min) & filtered_df.le(int32_max)).all(axis=1)]

In [62]:
df_filtered.shape

(2827860, 71)

In [69]:
# EXclude target columns if present
df_reduced = df_filtered.drop(columns=['Label', 'Lable_encoded'], errors='ignore')

# Compute the absolute correlation matridf_reduced
corr_matriX = df_reduced.corr().abs()

# Select the upper triangle of the correlation matriX
upper_tri = corr_matriX.where(np.triu(np.ones(corr_matriX.shape), k=1).astype(bool))

# Define a threshold for correlation; features with correlation higher than this will be dropped
threshold = 0.90

# Find the features to drop: any feature with a correlation greater than the threshold with another feature
to_drop = [column for column in upper_tri.columns if any(upper_tri[column] > threshold)]
print("Features to drop based on correlation threshold of", threshold, ":\n", to_drop)


Features to drop based on correlation threshold of 0.9 :
 ['Total Backward Packets', 'Total Length of Bwd Packets', 'Fwd Packet Length Std', 'Bwd Packet Length Mean', 'Bwd Packet Length Std', 'Flow IAT Max', 'Fwd IAT Total', 'Fwd IAT Mean', 'Fwd IAT Std', 'Fwd IAT Max', 'Bwd IAT Min', 'Fwd Packets/s', 'Max Packet Length', 'Packet Length Mean', 'Packet Length Std', 'Packet Length Variance', 'SYN Flag Count', 'CWE Flag Count', 'ECE Flag Count', 'Average Packet Size', 'Avg Fwd Segment Size', 'Avg Bwd Segment Size', 'Fwd Header Length.1', 'Subflow Fwd Packets', 'Subflow Fwd Bytes', 'Subflow Bwd Packets', 'Subflow Bwd Bytes', 'min_seg_size_forward', 'Active Min', 'Idle Mean', 'Idle Max', 'Idle Min']


In [70]:
len(to_drop)

32

In [71]:
# Drop the features from the DataFrame
df_reduced = df_reduced.drop(columns=to_drop,errors='ignored')

In [72]:
print("Original shape:", df_filtered.shape)
print("Reduced shape:", df_reduced.shape)

Original shape: (2827860, 71)
Reduced shape: (2827860, 38)


In [73]:
df_reduced['Lable_encoded'] = df_filtered['Lable_encoded'].values

In [74]:
df_reduced['Lable_encoded'].value_counts()

Lable_encoded
0    2271315
4     251748
5     158804
3     128025
2      13832
6       2180
1       1956
Name: count, dtype: int64

In [78]:
df_reduced.shape

(2827860, 39)

In [174]:
df_reduced.to_csv("reduced_dataset.csv",index=False)

In [76]:
# 7. Class Balancing (after feature selection)
min_class_size = df_reduced["Lable_encoded"].value_counts().min()

balanced_dfs = []
for label in df_reduced["Lable_encoded"].unique():
    class_subset = df_reduced[df_reduced["Lable_encoded"] == label]
    if len(class_subset) > min_class_size:
        class_sampled = resample(class_subset, replace=False, n_samples=min_class_size, random_state=42)
    else:
        class_sampled = class_subset
    balanced_dfs.append(class_sampled)

df_balanced = pd.concat(balanced_dfs).reset_index(drop=True)

In [77]:
df_balanced.shape

(13692, 39)

In [79]:
df_balanced

,Destination Port,Flow Duration,Total Fwd Packets,Total Length of Fwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Bwd Packet Length Max,Bwd Packet Length Min,Flow Bytes/s,Flow Packets/s,Flow IAT Mean,Flow IAT Std,Flow IAT Min,Fwd IAT Min,Bwd IAT Total,Bwd IAT Mean,Bwd IAT Std,Bwd IAT Max,Fwd PSH Flags,Fwd URG Flags,Fwd Header Length,Bwd Header Length,Bwd Packets/s,Min Packet Length,FIN Flag Count,RST Flag Count,PSH Flag Count,ACK Flag Count,URG Flag Count,Down/Up Ratio,Init_Win_bytes_forward,Init_Win_bytes_backward,act_data_pkt_fwd,Active Mean,Active Std,Active Max,Idle Std,Lable_encoded
0,49671,48,2,0,0,0,0.000000,0,0,0.000000e+00,41666.666670,48.000000,0.000000,48,48,0,0.0,0.0,0,0,0,40,0,0.000000,0,0,0,0,1,0,0,252,-1,0,0.0,0.0,0,0.0,0
1,49904,4,3,31,31,0,10.333333,0,0,7.750000e+06,750000.000000,2.000000,1.414214,1,1,0,0.0,0.0,0,0,0,96,0,0.000000,0,0,0,0,1,0,0,1307,-1,1,0.0,0.0,0,0.0,0
2,53,48698,2,88,44,44,44.000000,96,96,5.749723e+03,82.138897,16232.666667,28109.741236,3,3,4,4.0,0.0,4,0,0,40,40,41.069448,44,0,0,0,0,0,1,-1,-1,1,0.0,0.0,0,0.0,0
3,53,560,2,68,34,34,34.000000,93,93,4.535714e+05,7142.857143,186.666667,281.780647,2,2,47,47.0,0.0,47,0,0,64,64,3571.428571,34,0,0,0,0,0,1,-1,-1,1,0.0,0.0,0,0.0,0
4,80,24812,1,0,0,0,0.000000,0,0,0.000000e+00,80.606158,24812.000000,0.000000,24812,0,0,0.0,0.0,0,0,0,32,32,40.303079,0,0,0,0,1,1,1,254,70,0,0.0,0.0,0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13687,5811,78,1,0,0,0,0.000000,6,6,7.692308e+04,25641.025640,78.000000,0.000000,78,0,0,0.0,0.0,0,0,0,40,20,12820.512820,0,0,0,1,0,0,1,29200,0,0,0.0,0.0,0,0.0,5
13688,2200,42,1,2,2,2,2.000000,6,6,1.904762e+05,47619.047620,42.000000,0.000000,42,0,0,0.0,0.0,0,0,0,24,20,23809.523810,2,0,0,1,0,0,1,1024,0,0,0.0,0.0,0,0.0,5
13689,1236,43,1,0,0,0,0.000000,6,6,1.395349e+05,46511.627910,43.000000,0.000000,43,0,0,0.0,0.0,0,0,0,40,20,23255.813950,0,0,0,1,0,0,1,29200,0,0,0.0,0.0,0,0.0,5
13690,49156,46,1,0,0,0,0.000000,6,6,1.304348e+05,43478.260870,46.000000,0.000000,46,0,0,0.0,0.0,0,0,0,40,20,21739.130430,0,0,0,1,0,0,1,29200,0,0,0.0,0.0,0,0.0,5


In [80]:
df_balanced["Lable_encoded"].value_counts()

Lable_encoded
0    1956
2    1956
4    1956
6    1956
1    1956
3    1956
5    1956
Name: count, dtype: int64

In [81]:
df_balanced.to_csv("undersampled_datase.csv", index=False)

In [162]:
# 8. Separate final features and labels
y_final = df_balanced['Lable_encoded']
X_final = df_balanced.drop(columns=['Lable_encoded'])

In [163]:
# 9. Z-score Normalization
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_final)


In [165]:
# Final preprocessed dataset for GAN training
X_preprocessed = pd.DataFrame(X_scaled, columns=X_final.columns)
X_preprocessed['Label_encoded'] = y_final.values



In [166]:
X_preprocessed

,Destination Port,Flow Duration,Total Fwd Packets,Total Length of Fwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Bwd Packet Length Max,Bwd Packet Length Min,Flow Bytes/s,Flow Packets/s,Flow IAT Mean,Flow IAT Std,Flow IAT Min,Fwd IAT Min,Bwd IAT Total,Bwd IAT Mean,Bwd IAT Std,Bwd IAT Max,Fwd PSH Flags,Fwd URG Flags,Fwd Header Length,Bwd Header Length,Bwd Packets/s,Min Packet Length,FIN Flag Count,RST Flag Count,PSH Flag Count,ACK Flag Count,URG Flag Count,Down/Up Ratio,Init_Win_bytes_forward,Init_Win_bytes_backward,act_data_pkt_fwd,Active Mean,Active Std,Active Max,Idle Std,Label_encoded
0,3.393298,-0.462082,-0.212053,-0.105490,-0.165788,-0.131558,-0.159702,-0.491827,-0.254375,-0.055648,-0.068150,-0.335802,-0.444802,-0.046666,-0.076534,-0.273733,-0.169221,-0.207618,-0.218900,-0.224506,0.0,-0.238364,-0.374306,-0.227188,-0.316722,-0.199285,-0.008546,-1.113306,1.378267,-0.347995,-1.039358,-0.801365,-0.405716,-0.277634,-0.116992,-0.055657,-0.123754,-0.132138,0
1,3.411026,-0.462084,-0.152513,-0.101566,-0.132563,-0.131558,-0.117753,-0.491827,-0.254375,1.169936,3.206134,-0.335812,-0.444802,-0.046680,-0.076542,-0.273733,-0.169221,-0.207618,-0.218900,-0.224506,0.0,-0.130620,-0.374306,-0.227188,-0.316722,-0.199285,-0.008546,-1.113306,1.378267,-0.347995,-1.039358,-0.722349,-0.405716,-0.174052,-0.116992,-0.055657,-0.123754,-0.132138,0
2,-0.381889,-0.460484,-0.212053,-0.094351,-0.118630,1.130317,0.018921,-0.456242,2.555217,-0.054739,-0.260375,-0.332249,-0.441486,-0.046679,-0.076541,-0.273733,-0.169221,-0.207618,-0.218900,-0.224506,0.0,-0.238364,-0.270557,-0.226209,3.337569,-0.199285,-0.008546,-1.113306,-0.725549,-0.347995,0.710413,-0.820313,-0.405716,-0.174052,-0.116992,-0.055657,-0.123754,-0.132138,0
3,-0.381889,-0.462066,-0.212053,-0.096883,-0.129348,0.843527,-0.021675,-0.457354,2.467417,0.016080,-0.227737,-0.335772,-0.444769,-0.046680,-0.076541,-0.273731,-0.169213,-0.207618,-0.218897,-0.224506,0.0,-0.192188,-0.208308,-0.142020,2.507048,-0.199285,-0.008546,-1.113306,-0.725549,-0.347995,0.710413,-0.820313,-0.405716,-0.174052,-0.116992,-0.055657,-0.123754,-0.132138,0
4,-0.379835,-0.461269,-0.271593,-0.105490,-0.165788,-0.131558,-0.159702,-0.491827,-0.254375,-0.055648,-0.260382,-0.330365,-0.444802,-0.039417,-0.076542,-0.273733,-0.169221,-0.207618,-0.218900,-0.224506,0.0,-0.253756,-0.291307,-0.226227,-0.316722,-0.199285,-0.008546,-1.113306,1.378267,2.873605,0.710413,-0.801215,-0.398625,-0.277634,-0.116992,-0.055657,-0.123754,-0.132138,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13687,0.056208,-0.462081,-0.271593,-0.105490,-0.165788,-0.131558,-0.159702,-0.489603,-0.078775,-0.043483,-0.142229,-0.335796,-0.444802,-0.046657,-0.076542,-0.273733,-0.169221,-0.207618,-0.218900,-0.224506,0.0,-0.238364,-0.322432,0.078544,-0.316722,-0.199285,-0.008546,0.898226,-0.725549,-0.347995,0.710413,1.366727,-0.405616,-0.277634,-0.116992,-0.055657,-0.123754,-0.132138,5
13688,-0.218535,-0.462083,-0.271593,-0.105237,-0.163644,-0.074201,-0.151583,-0.489603,-0.078775,-0.025526,-0.040635,-0.335804,-0.444802,-0.046668,-0.076542,-0.273733,-0.169221,-0.207618,-0.218900,-0.224506,0.0,-0.269148,-0.322432,0.340601,-0.150618,-0.199285,-0.008546,0.898226,-0.725549,-0.347995,0.710413,-0.743545,-0.405616,-0.277634,-0.116992,-0.055657,-0.123754,-0.132138,5
13689,-0.291881,-0.462083,-0.271593,-0.105490,-0.165788,-0.131558,-0.159702,-0.489603,-0.078775,-0.033582,-0.045754,-0.335803,-0.444802,-0.046668,-0.076542,-0.273733,-0.169221,-0.207618,-0.218900,-0.224506,0.0,-0.238364,-0.322432,0.327396,-0.316722,-0.199285,-0.008546,0.898226,-0.725549,-0.347995,0.710413,1.366727,-0.405616,-0.277634,-0.116992,-0.055657,-0.123754,-0.132138,5
13690,3.354114,-0.462082,-0.271593,-0.105490,-0.165788,-0.131558,-0.159702,-0.489603,-0.078775,-0.035021,-0.059776,-0.335803,-0.444802,-0.046667,-0.076542,-0.273733,-0.169221,-0.207618,-0.218900,-0.224506,0.0,

In [167]:
# Find columns where all values are unique
unique_col=[]
for col in X_preprocessed.columns:
    if len(X_preprocessed[col].unique())==1:
        unique_col.append(col);
        

print(unique_col)

['Fwd URG Flags']


In [168]:
X_preprocessed  = X_preprocessed.drop(columns=unique_col)

In [173]:
X_preprocessed.to_csv("preprocessed_data1.csv", index=False)

In [172]:
X_preprocessed

,Destination Port,Flow Duration,Total Fwd Packets,Total Length of Fwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Bwd Packet Length Max,Bwd Packet Length Min,Flow Bytes/s,Flow Packets/s,Flow IAT Mean,Flow IAT Std,Flow IAT Min,Fwd IAT Min,Bwd IAT Total,Bwd IAT Mean,Bwd IAT Std,Bwd IAT Max,Fwd PSH Flags,Fwd Header Length,Bwd Header Length,Bwd Packets/s,Min Packet Length,FIN Flag Count,RST Flag Count,PSH Flag Count,ACK Flag Count,URG Flag Count,Down/Up Ratio,Init_Win_bytes_forward,Init_Win_bytes_backward,act_data_pkt_fwd,Active Mean,Active Std,Active Max,Idle Std,Label_encoded
0,3.393298,-0.462082,-0.212053,-0.105490,-0.165788,-0.131558,-0.159702,-0.491827,-0.254375,-0.055648,-0.068150,-0.335802,-0.444802,-0.046666,-0.076534,-0.273733,-0.169221,-0.207618,-0.218900,-0.224506,-0.238364,-0.374306,-0.227188,-0.316722,-0.199285,-0.008546,-1.113306,1.378267,-0.347995,-1.039358,-0.801365,-0.405716,-0.277634,-0.116992,-0.055657,-0.123754,-0.132138,0
1,3.411026,-0.462084,-0.152513,-0.101566,-0.132563,-0.131558,-0.117753,-0.491827,-0.254375,1.169936,3.206134,-0.335812,-0.444802,-0.046680,-0.076542,-0.273733,-0.169221,-0.207618,-0.218900,-0.224506,-0.130620,-0.374306,-0.227188,-0.316722,-0.199285,-0.008546,-1.113306,1.378267,-0.347995,-1.039358,-0.722349,-0.405716,-0.174052,-0.116992,-0.055657,-0.123754,-0.132138,0
2,-0.381889,-0.460484,-0.212053,-0.094351,-0.118630,1.130317,0.018921,-0.456242,2.555217,-0.054739,-0.260375,-0.332249,-0.441486,-0.046679,-0.076541,-0.273733,-0.169221,-0.207618,-0.218900,-0.224506,-0.238364,-0.270557,-0.226209,3.337569,-0.199285,-0.008546,-1.113306,-0.725549,-0.347995,0.710413,-0.820313,-0.405716,-0.174052,-0.116992,-0.055657,-0.123754,-0.132138,0
3,-0.381889,-0.462066,-0.212053,-0.096883,-0.129348,0.843527,-0.021675,-0.457354,2.467417,0.016080,-0.227737,-0.335772,-0.444769,-0.046680,-0.076541,-0.273731,-0.169213,-0.207618,-0.218897,-0.224506,-0.192188,-0.208308,-0.142020,2.507048,-0.199285,-0.008546,-1.113306,-0.725549,-0.347995,0.710413,-0.820313,-0.405716,-0.174052,-0.116992,-0.055657,-0.123754,-0.132138,0
4,-0.379835,-0.461269,-0.271593,-0.105490,-0.165788,-0.131558,-0.159702,-0.491827,-0.254375,-0.055648,-0.260382,-0.330365,-0.444802,-0.039417,-0.076542,-0.273733,-0.169221,-0.207618,-0.218900,-0.224506,-0.253756,-0.291307,-0.226227,-0.316722,-0.199285,-0.008546,-1.113306,1.378267,2.873605,0.710413,-0.801215,-0.398625,-0.277634,-0.116992,-0.055657,-0.123754,-0.132138,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13687,0.056208,-0.462081,-0.271593,-0.105490,-0.165788,-0.131558,-0.159702,-0.489603,-0.078775,-0.043483,-0.142229,-0.335796,-0.444802,-0.046657,-0.076542,-0.273733,-0.169221,-0.207618,-0.218900,-0.224506,-0.238364,-0.322432,0.078544,-0.316722,-0.199285,-0.008546,0.898226,-0.725549,-0.347995,0.710413,1.366727,-0.405616,-0.277634,-0.116992,-0.055657,-0.123754,-0.132138,5
13688,-0.218535,-0.462083,-0.271593,-0.105237,-0.163644,-0.074201,-0.151583,-0.489603,-0.078775,-0.025526,-0.040635,-0.335804,-0.444802,-0.046668,-0.076542,-0.273733,-0.169221,-0.207618,-0.218900,-0.224506,-0.269148,-0.322432,0.340601,-0.150618,-0.199285,-0.008546,0.898226,-0.725549,-0.347995,0.710413,-0.743545,-0.405616,-0.277634,-0.116992,-0.055657,-0.123754,-0.132138,5
13689,-0.291881,-0.462083,-0.271593,-0.105490,-0.165788,-0.131558,-0.159702,-0.489603,-0.078775,-0.033582,-0.045754,-0.335803,-0.444802,-0.046668,-0.076542,-0.273733,-0.169221,-0.207618,-0.218900,-0.224506,-0.238364,-0.322432,0.327396,-0.316722,-0.199285,-0.008546,0.898226,-0.725549,-0.347995,0.710413,1.366727,-0.405616,-0.277634,-0.116992,-0.055657,-0.123754,-0.132138,5
13690,3.354114,-0.462082,-0.271593,-0.105490,-0.165788,-0.131558,-0.159702,-0.489603,-0.078775,-0.035021,-0.059776,-0.335803,-0.444802,-0.046667,-0.076542,-0.273733,-0.169221,-0.207618,-0.218900,-0.224506,-0.238364,-0.322432,0.291228,-0.316722,-0.199285,-0.00

In [153]:
scaled_balenced_dataset['Lable_encoded'].value_counts()

Lable_encoded
0    1956
2    1956
4    1956
6    1956
1    1956
3    1956
5    1956
Name: count, dtype: int64